# Diffusion 

## Inference 4: ControlNet


## Workflow

#### Drive

If you need to load/save to your drive:

```python
import sys
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive/')

import os
os.chdir('drive/My Drive/IS53055B-DMLCP/DMLCP/python') # to change to another directory
```

#### Huggingface login

For some models and datasets, and if you want to push your model to HF (same as GitHub, but for models) you need to be logged into your HF account.

For that, you need to create an account [here](https://huggingface.co/) and then to ['/settings/tokens'](https://huggingface.co/settings/tokens) to create an access token.

```python
from pathlib import Path
from huggingface_hub import notebook_login
if not (Path.home()/'.huggingface'/'token').exists():
    notebook_login()
```

#### Install

1. On Colab, just use `pip` to install Huggingface libraries (see below).

2. Locally, the install is the same as the one used for Language models, see [`setup.md`](https://github.com/jchwenger/DMLCP/blob/main/setup.md#pytorch--huggingfacegradio).

In [ ]:
import sys

if 'google.colab' in sys.modules:
    !pip install --upgrade transformers diffusers accelerate

## Running stable diffusion
To run stable diffusion you will need to distinguish wether you are running this on a Mac M1/M2 or on Linux/Windows with a Nvidia GPU. 
On a M1/M2 Mac, diffusion will need a "warmup" phase to work properly (see [this link](https://huggingface.co/docs/diffusers/optimization/mps))

In [ ]:
import torch

# Get cpu, gpu or mps device for training.
# See: https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#creating-models
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import cv2
from skimage import feature

from diffusers import ControlNetModel
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionControlNetPipeline

Now this will load the pretrained model and download it the first time the cell is run (which might take a while).

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
).to(device)  # to accelerator

pipe.safety_checker = None # remove NSFW filter (too many innocuous prompts yield an empty image when I test it)
 # Note: this is no licence to do harm, it is to give *you* the responsibility
 # of your use. (Also, the HF safety_checker is very, very conservative, and rejects
 # a lot of abstract images.)

In [ ]:
prompt = "A cubist painting of the Star Treck character Spock, high quality, trending on artstation"

SAVE_MEMORY = False

if SAVE_MEMORY:                     # Saves memory at the cost of speed:
    pipe.enable_attention_slicing() #  https://huggingface.co/docs/diffusers/main/en/api/diffusion_pipeline#diffusers.DiffusionPipeline.enable_attention_slicing 

if device=="mps":                           # First-time "warmup" pass for M1/M2 macs
    _ = pipe(prompt, num_inference_steps=1) # https://huggingface.co/docs/diffusers/v0.4.1/en/optimization/mps

Generate the image (note increasing `num_inference_steps` will improve quality but be slower)

In [ ]:
image = pipe(prompt, num_inference_steps=20).images[0]

In [ ]:
plt.imshow(np.array(image))
plt.title(prompt)
plt.axis('off')
plt.show()

## Conditioning stable diffusion with ControlNet

[ControlNet](https://stablediffusionweb.com/ControlNet) ([repo](https://github.com/lllyasviel/ControlNet), [documentation](https://huggingface.co/docs/diffusers/api/pipelines/controlnet)) is a very recent and quite amazing advancement in image generation using stable diffusion. It allows conditioning the stable diffusion generation pipeline on an image input (similarly to pix2pix).

You also can see it running [as a space on Huggingface](https://huggingface.co/spaces/hysts/ControlNet-v1-1) for testing and forking.

In [ ]:
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16
)

Let's use skimage to create edges

In [ ]:
def apply_canny_skimage(img, sigma=1.5, invert=False):
    grey_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    edges = (feature.canny(grey_img, sigma=sigma)*255).astype(np.uint8)
    if invert:
        edges = cv2.bitwise_not(edges)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

# accessing the source image on Colab without connecting to the Drive
if 'google.colab' in sys.modules:
    import requests
    from io import BytesIO
    url = "https://raw.githubusercontent.com/jchwenger/DMLCP/main/python/images/spock.jpg"
    response = requests.get(url)
    img = np.array(Image.open(BytesIO(response.content)))
# if run locally, assuming you do so from the repo
else:
    img = plt.imread("images/spock.jpg") # can be any image
    
edges =  apply_canny_skimage(img)

# ControlNet expects a PIL Image as input
edges_image = Image.fromarray(edges)

plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(edges)
plt.show()

Setup the ControlNet model. Somehow converting to device takes ages on Mac.

In [ ]:
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    controlnet=controlnet, 
    safety_checker=None
).to(device) # to accelerator

In [ ]:
prompt = "A sculpture made of clay on a white background"

if SAVE_MEMORY:                     # Saves memory at the cost of speed:
    pipe.enable_attention_slicing() # https://huggingface.co/docs/diffusers/main/en/api/diffusion_pipeline#diffusers.DiffusionPipeline.enable_attention_slicing 

if device=='mps':                           # First-time "warmup" pass for M1/M2 macs
    _ = pipe(prompt, num_inference_steps=1) # https://huggingface.co/docs/diffusers/v0.4.1/en/optimization/mps

In [ ]:
image = pipe(
    prompt,
    image=edges_image,
    num_inference_steps=30,
).images[0]

plt.imshow(np.array(image))
plt.title(prompt)
plt.axis('off')
plt.show()